# Atlas 14 Spatial Variance Analysis

This notebook demonstrates how to analyze spatial variability of NOAA Atlas 14 precipitation frequency estimates within a HEC-RAS model domain. This helps determine whether uniform rainfall assumptions are appropriate for rain-on-grid modeling.

**Key Features:**
- Remote access to NOAA Atlas 14 CONUS data via HTTP range requests (99.9% data reduction)
- Integration with HEC-RAS 2D flow area extents
- Variance statistics (min, max, mean, range) for precipitation frequency
- Decision support for uniform vs. spatially distributed rainfall

**When to use this analysis:**
- Before rain-on-grid modeling to assess uniformity assumptions
- For large model domains (>100 sq mi) where spatial variance may be significant
- When comparing multiple design storm events

In [ ]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = True  # <-- TOGGLE THIS

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

## 1. Direct Bounds Query

First, let's demonstrate querying Atlas 14 data for a specific geographic area without a HEC-RAS project. This is useful for exploring data availability and understanding precipitation patterns.

In [ ]:
from ras_commander.precip import Atlas14Grid

# Query Atlas 14 data for Houston, TX area
# This uses HTTP range requests to download only the needed data
pfe = Atlas14Grid.get_pfe_for_bounds(
    bounds=(-95.5, 29.5, -95.0, 30.0),  # (west, south, east, north)
    durations=[6, 12, 24],
    return_periods=[10, 50, 100],
    buffer_percent=0
)

print(f"Bounds: {pfe['bounds']}")
print(f"Grid dimensions: {pfe['lat'].shape[0]} x {pfe['lon'].shape[0]}")
print(f"Durations: {pfe['durations']} hours")
print(f"Return periods: {pfe['return_periods']} years")

In [ ]:
import numpy as np

# Examine the 100-year, 24-hour precipitation
data_24hr = pfe['pfe_24hr']
ari = pfe['ari']

# Find index for 100-year (ARI dimension is last)
ari_100_idx = np.argmin(np.abs(ari - 100))

data_100yr_24hr = data_24hr[:, :, ari_100_idx]

print(f"100-year, 24-hour precipitation:")
print(f"  Min: {np.nanmin(data_100yr_24hr):.2f} inches")
print(f"  Max: {np.nanmax(data_100yr_24hr):.2f} inches")
print(f"  Mean: {np.nanmean(data_100yr_24hr):.2f} inches")
print(f"  Range: {np.nanmax(data_100yr_24hr) - np.nanmin(data_100yr_24hr):.2f} inches")
print(f"  Range %: {(np.nanmax(data_100yr_24hr) - np.nanmin(data_100yr_24hr)) / np.nanmin(data_100yr_24hr) * 100:.1f}%")

## 2. Visualize Spatial Variability

In [ ]:
import matplotlib.pyplot as plt

# Create a simple visualization of spatial variability
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, duration in enumerate([6, 12, 24]):
    data = pfe[f'pfe_{duration}hr'][:, :, ari_100_idx]
    
    ax = axes[idx]
    im = ax.imshow(data, origin='lower', 
                   extent=[pfe['lon'].min(), pfe['lon'].max(),
                          pfe['lat'].min(), pfe['lat'].max()],
                   cmap='Blues')
    plt.colorbar(im, ax=ax, label='Depth (inches)')
    ax.set_title(f'100-yr, {duration}-hr')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

plt.suptitle('Atlas 14 Precipitation Frequency - Houston Area', fontsize=12)
plt.tight_layout()
plt.show()

## 3. Point Query for Single Location

For quick lookups at a specific location:

In [ ]:
# Get PFE for a single point (downtown Houston)
point_df = Atlas14Grid.get_point_pfe(
    lat=29.76,
    lon=-95.37,
    durations=[6, 12, 24],
    return_periods=[10, 25, 50, 100, 500]
)

print("Point Precipitation Frequency Estimates (Houston, TX):")
point_df_pivot = point_df.pivot(
    index='duration_hr',
    columns='return_period_yr',
    values='depth_inches'
)
print(point_df_pivot.round(2))

## 4. Integration with HEC-RAS Project

The most powerful feature is automatic integration with HEC-RAS projects. The `get_pfe_from_project()` method extracts the extent from your geometry HDF file and downloads only the needed data.

**Note:** This example uses a sample project. Replace with your actual project path.

In [ ]:
from ras_commander import RasExamples, init_ras_project

# Extract an example project with 2D flow areas
project_path = RasExamples.extract_project("BaldEagleCrkMulti2D", suffix="atlas14_variance")
print(f"Project extracted to: {project_path}")

# Initialize project
ras = init_ras_project(project_path, hecras_version="6.6")
print(f"\nProject: {ras.project_name}")
print(f"Geometry files: {len(ras.geom_df)}")

In [ ]:
from pathlib import Path

# Find the geometry HDF file
geom_hdfs = list(project_path.glob("*.g*.hdf"))
if geom_hdfs:
    geom_hdf = geom_hdfs[0]
    print(f"Geometry HDF: {geom_hdf.name}")
else:
    print("No geometry HDF found")
    geom_hdf = None

In [ ]:
if geom_hdf:
    # Get Atlas 14 data using 2D flow area extent
    pfe_project = Atlas14Grid.get_pfe_from_project(
        geom_hdf=geom_hdf,
        extent_source="2d_flow_area",  # Use 2D flow areas
        durations=[6, 12, 24],
        return_periods=[10, 25, 50, 100],
        buffer_percent=10.0,
        ras_object=ras
    )
    
    print(f"Extent source: {pfe_project['extent_source']}")
    print(f"Bounds: {pfe_project['bounds']}")
    print(f"Grid dimensions: {pfe_project['lat'].shape[0]} x {pfe_project['lon'].shape[0]}")
    if 'mesh_area_names' in pfe_project:
        print(f"2D Areas: {pfe_project['mesh_area_names']}")

## 5. Full Variance Analysis

The `Atlas14Variance` class provides comprehensive variance analysis:

In [ ]:
from ras_commander.precip import Atlas14Variance

if geom_hdf:
    # Run full variance analysis
    results = Atlas14Variance.analyze(
        geom_hdf=geom_hdf,
        durations=[6, 12, 24],
        return_periods=[10, 25, 50, 100],
        extent_source="2d_flow_area",
        variance_denominator='min',  # range_pct = (max-min)/min * 100
        ras_object=ras
    )
    
    print("\nVariance Analysis Results:")
    print(results.to_string(index=False))

In [ ]:
if geom_hdf and not results.empty:
    # Check if uniform rainfall is appropriate
    ok, message = Atlas14Variance.is_uniform_rainfall_appropriate(results, threshold_pct=10.0)
    
    print(f"\n{'✓' if ok else '⚠️'} {message}")
    
    # Show highest variance events
    if not ok:
        print("\nHighest variance events:")
        high_var = results.nlargest(3, 'range_pct')[['duration_hr', 'return_period_yr', 'min_inches', 'max_inches', 'range_pct']]
        print(high_var.to_string(index=False))

## 6. Quick Variance Check

For a rapid assessment using a single representative event (default: 100-year, 24-hour):

In [ ]:
if geom_hdf:
    # Quick check for 100-year, 24-hour
    quick_stats = Atlas14Variance.analyze_quick(
        geom_hdf=geom_hdf,
        duration=24,
        return_period=100
    )
    
    print("Quick Variance Check (100-yr, 24-hr):")
    print(f"  Min: {quick_stats['min']:.2f} inches")
    print(f"  Max: {quick_stats['max']:.2f} inches")
    print(f"  Mean: {quick_stats['mean']:.2f} inches")
    print(f"  Range: {quick_stats['range']:.2f} inches")
    print(f"  Range %: {quick_stats['range_pct']:.1f}%")
    
    if quick_stats['range_pct'] > 10:
        print("\n⚠️ Consider using spatially variable rainfall")
    else:
        print("\n✓ Uniform rainfall is likely appropriate")

## 7. Generate Report

Export results with plots for engineering review:

In [ ]:
if geom_hdf and not results.empty:
    output_dir = project_path / "Atlas14_Variance_Report"
    
    report_dir = Atlas14Variance.generate_report(
        results_df=results,
        output_dir=output_dir,
        project_name="Bald Eagle Creek",
        include_plots=True
    )
    
    print(f"\nReport generated at: {report_dir}")
    print("\nFiles created:")
    for f in report_dir.glob("*"):
        print(f"  - {f.name}")

## 8. Cleanup

In [ ]:
# Clear the coordinate cache to free memory
Atlas14Grid.clear_cache()
print("Cache cleared")

# Optionally clean up extracted project
# import shutil
# shutil.rmtree(project_path.parent, ignore_errors=True)

## Summary

This notebook demonstrated:

1. **Direct Bounds Query** - Get Atlas 14 data for any geographic area
2. **Point Query** - Quick lookup for a single location
3. **HEC-RAS Integration** - Automatic extent extraction from 2D flow areas
4. **Variance Analysis** - Calculate min/max/mean/range statistics
5. **Decision Support** - Determine if uniform rainfall is appropriate
6. **Report Generation** - Export results for engineering review

**Key Takeaways:**
- Range percentage > 10% suggests spatially variable rainfall should be considered
- The analysis uses HTTP range requests to minimize data transfer (~99.9% reduction)
- All CONUS locations are supported (24°N-50°N, -125°W to -66°W)

**References:**
- [NOAA Atlas 14](https://hdsc.nws.noaa.gov/pfds/)
- [Precipitation Frequency Data Server](https://hdsc.nws.noaa.gov/pfds/pfds_gis.html)